# Target Offloading

## Overview

We start by reviewing a short presentation.
Execute the cell below to display it or download and open it locally.

In [ ]:
%%HTML
<div align="center"><iframe src="slides/Introduction-to-GPU-Computing.pdf" width=800 height=500 frameborder=0></iframe></div>


In contrast to other GPU programming approaches, OpenMP follows a slightly different naming convention ([OpenMP 5.1 - 1.2.1](https://www.openmp.org/spec-html/5.1/openmpsu1.html)).
* A *device* is 'an implementation-defined logical execution engine'.
* The *host*, or *initial device*, is the device on which the OpenMP program begins execution.
* The *target* is a device onto which code and data may be offloaded from the host device. In many cases it is a GPU, but it doesn't have to be.

In [ ]:
%load_ext ice.magic

## Target

The `target` construct transfers execution to the device (target) ([OpenMP 5.1 - 2.14.5](https://www.openmp.org/spec-html/5.1/openmpsu68.html)).
\
Everything within the target block is executed on the target.
Only a sub-set of features is available, e.g. no `std::cout`. If you are familiar with other GPU programming approaches -- similar restrictions apply with OpenMP as well.

In [ ]:
%%cpp_omp_target -o code/target/hello-world.cpp

std::cout << "Hello from the CPU" << std::endl;

#pragma omp target
{           ☝
    printf("Hello from the GPU\n");
} //# implicit synchronization with the target

### Compilation

You might have noticed that we used a different magic this time -- `cpp_omp_target`.
\
This switches to a different compiler, `nvc++`, and adds a different set of flags.
As before, using the `-v` switch displays more detailed information.
\
Adding the optional `-Minfo=mp` compiler flag triggers the compiler to emit information about how the application is mapped to the target.

In [ ]:
%%cpp_omp_target -o code/target/hello-world.cpp -f Minfo=mp

std::cout << "Hello from the CPU" << std::endl;

#pragma omp target
{
    printf("Hello from the GPU\n");
} //# implicit synchronization with the target

### Checking for Host Execution

In some cases it might be necessary to programmatically check whether execution is currently on the host or on the device, e.g. when using multiple nested functions.
`omp_is_initial_device` can be used to perform that check ([OpenMP 5.1 - 3.7.6](https://www.openmp.org/spec-html/5.1/openmpsu166.html)).

In [ ]:
%%cpp_omp_target -o code/target/initial-device.cpp

std::cout << omp_is_initial_device() << std::endl;
             ☝

#pragma omp target
{
    printf("%d\n", omp_is_initial_device());
                   ☝
} 

## Parallel Execution

So far, everything in our `target` region has been executed serially since the target construct doesn't generate parallelism.
\
In the following steps, we will add hierarchical parallelism (to match the GPU architecture discussed before), and workload sharing.
\
We will use the the following example as baseline.
In it, the loop is executed on the device, but only with a single thread.

In [ ]:
%%cpp_omp_target -o code/target/target-serial.cpp

#pragma omp target
for (auto i = 0; i < 10; ++i)
    printf("%d\n", i);

### Teams

`teams` construct generates a *league of teams* ([OpenMP 5.1 - 2.7](https://www.openmp.org/spec-html/5.1/openmpse15.html)).
\
A team is comparable but not necessarily identical to a CUDA thread block.
Each team initially has only one thread and each team executes the same code.
The number of teams can be *limited* with `num_teams` -- this sets an upper bound, not the exact number.

The id of the current team can be querried with `omp_get_team_num` ([OpenMP 5.1 - 3.4.2](https://www.openmp.org/spec-html/5.1/openmpsu152.html)).
\
`omp_get_thread_num` returns the current thread id *within the current team*.

In [ ]:
%%cpp_omp_target -o code/target/target-teams.cpp

#pragma omp target
#pragma omp teams num_teams(2)
            ☝   ☝
    printf("Team %d, thread %d\n", omp_get_team_num(), omp_get_thread_num());
                                   ☝

### Parallel

`parallel` generates a parallel region with multiple threads per team.
The number of threads per team can be limited with `thread_limit`

In [ ]:
%%cpp_omp_target -o code/target/thread-limit.cpp

#pragma omp target
#pragma omp teams parallel num_teams(2) thread_limit(2)
                  ☝                    ☝
    printf("Team %d, thread %d\n", omp_get_team_num(), omp_get_thread_num());

### Distribute

For loops, worksharing constructs are required additionally.

`distribute` distributes the iteration space across teams ([OpenMP 5.1 - 2.11.6](https://www.openmp.org/spec-html/5.1/openmpsu50.html)).
\
Schedules can be specified using `dist_schedule`.

In [ ]:
%%cpp_omp_target -o code/target/target-distribute.cpp

#pragma omp target
#pragma omp teams num_teams(2)
#pragma omp distribute
            ☝
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

### For

`for` distributes the *team's* iteration space over the team's threads.

In [ ]:
%%cpp_omp_target -o code/target/target-for.cpp

#pragma omp target
#pragma omp teams num_teams(2)
#pragma omp distribute parallel for
                                ☝
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

### SIMD

Additionally, the `simd` construct is also available.
What exactly is mapped how is compiler dependent.
For NVIDIA, *usually* teams are mapped to CUDA thread blocks, threads are mapped to CUDA threads and simd is ignored.

In [ ]:
%%cpp_omp_target -o code/target/target-simd.cpp

#pragma omp target teams distribute parallel for simd
                                                 ☝
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

## Collapsing Loops

Similar to how loops are handled on CPU, `collapse` can be used to merge multiple loops in a perfect nest.
This is especially important on GPUs since massive parallelism is required to fully utilize the hardware.

In [ ]:
%%cpp_omp_target -o code/target/collapse.cpp

#pragma omp target teams distribute parallel for simd collapse(2)
                                                      ☝
for (auto i = 0; i < 2; ++i)
    for (auto j = 0; j < 5; ++j)
        printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i * 5 + j);

## Target Data

Target regions span their own data environment.
In addition to the clauses already discussed in the [data environment](data-environment.ipynb) notebook, target data mapping clauses are available.
Consider the following example:

In [ ]:
%%cpp_omp_target -o code/target/map.cpp

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target teams distribute parallel for map(tofrom: vec[0:10])
                                                 ☝
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

Here, the contents of `vec` are copied to the target when entering the target region and copied back to the host when leaving it.
Available map types in the `map` clause are
* `to` which copies data from host to target,
* `from` which copies data from target to host,
* `tofrom` which combines the behavior of to and from, and
* `alloc` which allocates data on the target but does not initialize it.

### Implicit Behavior

Target data environments implement the following implicit behavior if not specified otherwise:
* Scalar variables are `firstprivate`
    * They are copied to the device and each thread has its own version
    * Changes are neither synchronized between threads nor copied back to the host

In [ ]:
%%cpp_omp_target -o code/target/implicit-data.cpp

int a = 10;
   ☝
static int b = 20;
          ☝

#pragma omp target teams parallel
if (0 == omp_get_team_num() && 0 == omp_get_thread_num()) {
    printf("a = %d, b = %d\n", a, b);
    a *= 10;
    b *= 10;
}

printf("a = %d, b = %d\n", a, b);

* Statically allocated arrays are treated as `map(tofrom)`

In [ ]:
%%cpp_omp_target -o code/target/static-array.cpp

int vec[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
    ☝

#pragma omp target teams distribute parallel for
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (const auto& val : vec)
    std::cout << val << " ";
std::cout << std::endl;

* Dynamically allocated arrays need to be mapped explicitly.

In [ ]:
%%cpp_omp_target -o code/target/missing-map.cpp

int *vec = new int[10];
    ☝
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target teams distribute parallel for
//# due to the missing map clause, this example will not work
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

### Target Data Region

Mapping data for each target region individually can generate some drawbacks (code bloating, performance issues).
OpenMP offers `target data` constructs as an alternative ([OpenMP 5.1 - 2.14.2](https://www.openmp.org/spec-html/5.1/openmpsu65.html)).

In [ ]:
%%cpp_omp_target -o code/target/target-data.cpp

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target data map(tofrom: vec[0:10])
{                 ☝
    #pragma omp target teams distribute parallel for
    for (auto i = 0; i < 10; ++i)
        vec[i] *= 2;

    #pragma omp target teams distribute parallel for
    for (auto i = 0; i < 10; ++i)
        vec[i] *= 2;
}

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

If the software architecture requires a more unstructured approach `target enter data` ([OpenMP 5.1 - 2.14.3](https://www.openmp.org/spec-html/5.1/openmpsu66.html)) and `target exit data` ([OpenMP 5.1 - 2.14.4](https://www.openmp.org/spec-html/5.1/openmpsu67.html)) are available.
These can be helpful when performing the mapping in separate functions, e.g. in the constructor and destructor of a class.

In [ ]:
%%cpp_omp_target -o code/target/enter-data.cpp

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target enter data map(to: vec[0:10])
                  ☝

#pragma omp target teams distribute parallel for
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

#pragma omp target exit data map(from: vec[0:10])
                  ☝

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

## An Aside on Managed Memory

On newer architecture, managed memory or unified memory is available as an alternative (see, e.g., this [blog post](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/)).
When used, allocations are done as managed memory and all transfers between host and target are done implicitly.
\
In OpenMP it is activated by adding the compiler flag `-gpu=managed` and by specifying `#pragma omp requires unified_shared_memory`.

In [ ]:
%%cpp_omp_target -o code/target/managed-mem.cpp -v -f gpu=managed
                                                   ☝

#pragma omp requires unified_shared_memory
           ☝

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target teams distribute parallel for // no map clauses
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

## Reductions



In [ ]:
%%cpp_omp_target -o code/target/reduction.cpp

auto sum = 0;

#pragma omp target teams distribute parallel for reduction( + : sum )
                                                 ☝
for (auto i = 0; i < 100; ++i)
    sum += i;

std::cout << "Sum is " << sum << std::endl;